In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import holidays

from gluonts.dataset.multivariate_grouper import MultivariateGrouper
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split

from uni2ts.eval_util.plot import plot_single, plot_next_multi
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule

from neuralforecast.core import NeuralForecast
from neuralforecast.models import NHITS, NBEATSx, TSMixerx

from neuralforecast.losses.numpy import mae, mse

import warnings
warnings.filterwarnings('ignore')

In [2]:
month = pd.read_csv('~/envs/uni2ts/data/month_clean.csv')
week = pd.read_csv('~/envs/uni2ts/data/week_clean.csv')
day = pd.read_csv('~/envs/uni2ts/data/day_clean.csv')
hour = pd.read_csv('~/envs/uni2ts/data/hour_clean.csv')

In [3]:
import holidays

m_date_range = pd.date_range(start=month.iloc[0]['ds'], end=month.iloc[-1]['ds'], freq='D')
w_date_range = pd.date_range(start=week.iloc[0]['ds'], end=week.iloc[-1]['ds'], freq='D')
d_date_range = pd.date_range(start=day.iloc[0]['ds'], end=day.iloc[-1]['ds'], freq='D')
h_date_range = pd.date_range(start=hour.iloc[0]['ds'], end=hour.iloc[-1]['ds'], freq='D')

us_holidays = holidays.US()


In [4]:
def calculate_owa(model_sMAPE, model_MASE, naive_sMAPE, naive_MASE):
    owa = 0.5 * ((model_sMAPE / naive_sMAPE) + (model_MASE / naive_MASE))
    return owa

In [5]:
month.insert(0, 'unique_id', 'M')
week.insert(0, 'unique_id', 'W')
day.insert(0, 'unique_id', 'D')
hour.insert(0, 'unique_id', 'H')

In [ ]:
hour.insert(0, 'unique_id', 'H')

In [7]:
alxm = pd.DataFrame(month, columns=['unique_id', 'ds', 'alx'])
alxm.rename(columns={'alx':'y'}, inplace=True)
alxm['is_holiday'] = alxm['ds'].apply(lambda x: 1 if x in us_holidays else 0)
alxm['ds'] = pd.to_datetime(alxm['ds'])
tr_alxm = alxm[:48]
ts_alxm = alxm[48:]

gcom = pd.DataFrame(month, columns=['unique_id', 'ds', 'gco'])
gcom.rename(columns={'gco':'y'}, inplace=True)
gcom['is_holiday'] = gcom['ds'].apply(lambda x: 1 if x in us_holidays else 0)
gcom['ds'] = pd.to_datetime(gcom['ds'])
tr_gcom = gcom[:48]
ts_gcom = gcom[48:]

mcom = pd.DataFrame(month, columns=['unique_id', 'ds', 'mco'])
mcom.rename(columns={'mco':'y'}, inplace=True)
mcom['is_holiday'] = mcom['ds'].apply(lambda x: 1 if x in us_holidays else 0)
mcom['ds'] = pd.to_datetime(mcom['ds'])
tr_mcom = mcom[:48] 
ts_mcom = mcom[48:]

alxw = pd.DataFrame(week, columns=['unique_id', 'ds', 'alx'])
alxw.rename(columns={'alx':'y'}, inplace=True)
alxw['is_holiday'] = alxw['ds'].apply(lambda x: 1 if x in us_holidays else 0)
alxw['ds'] = pd.to_datetime(alxw['ds'])
tr_alxw = alxw[:209]
ts_alxw = alxw[209:]

gcow = pd.DataFrame(week, columns=['unique_id', 'ds', 'gco'])
gcow.rename(columns={'gco':'y'}, inplace=True)
gcow['is_holiday'] = gcow['ds'].apply(lambda x: 1 if x in us_holidays else 0)
gcow['ds'] = pd.to_datetime(gcow['ds'])
tr_gcow = gcow[:209]
ts_gcow = gcow[209:]

mcow = pd.DataFrame(week, columns=['unique_id', 'ds', 'mco'])
mcow.rename(columns={'mco':'y'}, inplace=True)
mcow['is_holiday'] = mcow['ds'].apply(lambda x: 1 if x in us_holidays else 0)
mcow['ds'] = pd.to_datetime(mcow['ds'])
tr_mcow = mcow[:209]
ts_mcow = mcow[209:]

alxd = pd.DataFrame(day, columns=['unique_id', 'ds', 'alx'])
alxd.rename(columns={'alx':'y'}, inplace=True)
alxd['is_holiday'] = alxd['ds'].apply(lambda x: 1 if x in us_holidays else 0)
alxd['ds'] = pd.to_datetime(alxd['ds'])
tr_alxd = alxd[:365]
ts_alxd = alxd[365:]

gcod = pd.DataFrame(day, columns=['unique_id', 'ds', 'gco'])
gcod.rename(columns={'gco':'y'}, inplace=True)
gcod['is_holiday'] = gcod['ds'].apply(lambda x: 1 if x in us_holidays else 0)
gcod['ds'] = pd.to_datetime(gcod['ds'])
tr_gcod = gcod[:365]
ts_gcod = gcod[365:]

mcod = pd.DataFrame(day, columns=['unique_id', 'ds', 'mco'])
mcod.rename(columns={'mco':'y'}, inplace=True)
mcod['is_holiday'] = mcod['ds'].apply(lambda x: 1 if x in us_holidays else 0)
mcod['ds'] = pd.to_datetime(mcod['ds'])
tr_mcod = mcod[:365]
ts_mcod = mcod[365:]

alxh = pd.DataFrame(hour, columns=['unique_id', 'ds', 'alx'])
alxh.rename(columns={'alx':'y'}, inplace=True)
alxh['is_holiday'] = alxh['ds'].apply(lambda x: 1 if x in us_holidays else 0)
alxh['ds'] = pd.to_datetime(alxh['ds'])
tr_alxh = alxh[:2184]
ts_alxh = alxh[2184:]

gcoh = pd.DataFrame(hour, columns=['unique_id', 'ds', 'gco'])
gcoh.rename(columns={'gco':'y'}, inplace=True)
gcoh['is_holiday'] = gcoh['ds'].apply(lambda x: 1 if x in us_holidays else 0)
gcoh['ds'] = pd.to_datetime(gcoh['ds'])
tr_gcoh = gcoh[:2184]
ts_gcoh = gcoh[2184:]

mcoh = pd.DataFrame(hour, columns=['unique_id', 'ds', 'mco'])
mcoh.rename(columns={'mco':'y'}, inplace=True)
mcoh['is_holiday'] = mcoh['ds'].apply(lambda x: 1 if x in us_holidays else 0)
mcoh['ds'] = pd.to_datetime(mcoh['ds'])
tr_mcoh = mcoh[:2184]
ts_mcoh = mcoh[2184:]

In [8]:
moirai_alxm = alxm.set_index('ds')
moirai_gcom = gcom.set_index('ds')
moirai_mcom = mcom.set_index('ds')
moirai_alxw = alxw.set_index('ds')
moirai_gcow = gcow.set_index('ds')
moirai_mcow = mcow.set_index('ds')
moirai_alxd = alxd.set_index('ds')
moirai_gcod = gcod.set_index('ds')
moirai_mocd = mcod.set_index('ds')
moirai_alxh = alxh.set_index('ds')
moirai_gcoh = gcoh.set_index('ds')
moirai_mcoh = mcoh.set_index('ds')

In [9]:
def get_median_and_ci(data, horizon, id, confidence=0.80):
    n_samples, n_timesteps = data.shape

    # Calculate the median for each timestep
    medians = np.median(data, axis=0)

    # Calculate teh lower and upper percentail for the given confidence interval
    lower_percentile = (1 - confidence) / 2 * 100
    upper_percentile = (1 + confidence) / 2 * 100

    # Calculate the lower and upper bounds for each timestep
    lower_bounds = np.percentile(data, lower_percentile, axis=0)
    upper_bounds = np.percentile(data, upper_percentile, axis=0)

    # Create a DataFrame with the results
    df = pd.DataFrame({
        'unique_id': id,
        'Moriai': medians,
        f'Moirai-lo-{int(confidence*100)}': lower_bounds,
        f'Moirai-hi-{int(confidence*100)}': upper_bounds
    })

    return df

In [10]:
alxm_ds = PandasDataset.from_long_dataframe(
    moirai_alxm,
    target='y',
    item_id='unique_id',
    feat_dynamic_real = ['is_holiday']
),

test_size = 12
horizon = 12

tr_alxm_ds, ts_alxm_ds = split(
    alxm_ds, offset=-test_size
)

ts_alxm_data = ts_alxm_ds.generate_instances(
    prediction_length=horizon,
    windows=test_size//horizon,
    distance=horizon
)

alxm_model = MoiraiForecast(
    module=MoiraiModule.from_pretrained(f"Salesforecs/moirai-1.0-R-small"),
    prediction_length=horizon,
    context_length=500,
    patch_size="auto",
    num_samples=100,
    target_dim=1,
    feat_dynamic_real_dim=alxm_ds.num_feat_dynamic_real,
    past_feat_dynamic_real_dim=alxm_ds.num_past_feat_dynamic_real
)

alxm_predictor = alxm_model.create_predictor(batch_size=32)
alxm_forecasts = alxm_predictor.predict(ts_alxm_ds.input)
alxm_forecasts = list(forecasts)

alxm_moirai_preds = [
    get_median_and_ci(
        data=alxm_forecasts[i].samples,
        horizon = horizon,
        id = 1
    )
    for i in range(12)
]

alxm_moirai_preds_df = pd.concat(alxm_moirai_preds, axis=0, ignore_index=True)

TypeError: MoiraiModule.__init__() missing 7 required positional arguments: 'distr_output', 'd_model', 'num_layers', 'patch_sizes', 'max_seq_len', 'attn_dropout_p', and 'dropout_p'